1. Scan your notes at 300 dpi. This is the absolute best quality that I would recommend for handwritten notes. I wanted to capture every detail of my notes and thought "well, let's use 600 dpi." That was a big mistake on my part. Your handwritten notes don't need that level of detail. Eventually you will have to compress them significantly since pdf readers and notetaking apps eventually crap out at huge pdf sizes. Save your time **Scan at 300 dpi and A4/US Letter (Portrait/Landscape)**. Also scanning at ultra high-resolution or sizes will make majority of the tools and libraries think that you are running a DeCompression Bomb DOS Attack.

2. Since I was a smart ass and wanted to scan at 600 dpi and not use any app like Office Lens, Adobe Scan etc. the entirity of the next steps are required. This led to a lot of self-reflection with questions like "why am I such a geek?", "why do I have to make my life difficult so much?", "Am I a masochist?". I could have just used the apps like a normie and be done with this 2 years ago! But I guess it is too late to regret.

3. Now the problem of being a smartass is that you have learn how to navigate on your own. So began the search for an app which could mimic the functionality offerred by existing apps. If nothing like this existed my hope was to read papers and implement similar systems on my own. Thankfully, I found out about [Noteshrink](https://github.com/mzucker/noteshrink) which was exclusivly created for the purpose of shrinking scanned handwritten notes efficiently. For a while this seemed to solve my issue. However, I wanted to make sure that all of my notes get converted correctly. So I picked up a random 5% of my notes and tried to convert it to a pdf. However, I noticed that there were issues post noteshrinking in a few of them which pushed me down a rabbit hole to solve this problem. Also, want dracula themed notes, so dug myself a deeper grave.

You will need `noteshrink`, `img2pdf` (since `convert` craps out a larger pdf sizes unless you change the policy) and `pdftoppm`

In [ ]:
import subprocess, os, shutil, glob, time
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from preprocess import process_image

### Directory Management

Place the PDF you want to convert in `./upload`.
The following program will create a folder named archival.

In [ ]:
def check_pdf_size(pdf_path):
    """Verify that the PDF file size is less than 2 GB."""
    max_size = 2 * 1024 * 1024 * 1024  # 2 GB in bytes
    if pdf_path.stat().st_size > max_size:
        print(f"PDF larger than 2 GBs: {pdf_path}")
        return False
    return True

def setup_directories(pdf_path, root_archival):
    """Create necessary directories for the archival process."""
    pdf_name = pdf_path.stem
    base_dir = root_archival / pdf_name
    base_dir.mkdir(parents=True, exist_ok=True)
    (base_dir / "original_images").mkdir(exist_ok=True)
    (base_dir / "preprocessed_images").mkdir(exist_ok=True)
    (base_dir / "noteshrinked_images").mkdir(exist_ok=True)
    (base_dir / "noteshrinked_post_processed_images").mkdir(exist_ok=True)
    return base_dir

def convert_pdf_to_png(pdf_path, output_dir, dpi=300):
    """Convert PDF to PNG using pdftoppm."""
    output_prefix = output_dir / pdf_path.stem
    print("Converting PDF to PNG")
    command = ' '.join(["pdftoppm", "-png", "-progress", "-r", str(dpi), str(pdf_path), str(output_prefix)])
    subprocess.run(command, shell=True)

def run_noteshrink(preprocessed_images_dir):
    """Run the noteshrink command on processed images."""
    files = [f for f in os.listdir(preprocessed_images_dir) if f.endswith('.png') and 'ns_page' not in f]
    print(files)
    cmd = ' '.join(['python',
        noteshrink_py_path, '-w', '-n 5', '-b ns_page', '-C', '--dracula', '-o noteshrinked.pdf',
        '-c "img2pdf --verbose %i --output %o"', ' '.join(files)
    ])
    subprocess.run(cmd, shell=True, cwd=preprocessed_images_dir)

def process_pdf(pdf_path, root_archival, upload_dir):
    """Process a single PDF file."""
    pdf_path = Path(pdf_path)  # Ensure pdf_path is a Path object
    if not check_pdf_size(pdf_path):
        return
    base_dir = setup_directories(pdf_path, root_archival)
    shutil.copy(str(pdf_path), str(base_dir))
    convert_pdf_to_png(base_dir / pdf_path.name, base_dir / "original_images")
    # Process each image in the original_images directory
    original_images_dir = base_dir / "original_images"
    preprocessed_images_dir = base_dir / "preprocessed_images"
    for image_file in original_images_dir.iterdir():
        if image_file.suffix.lower() == '.png':
            process_image(image_file.name, original_images_dir, preprocessed_images_dir)
    run_noteshrink(base_dir / "preprocessed_images")

def main(upload_dir, archival_dir):
    """Main function to process all PDFs in the upload directory."""
    root_archival = Path(archival_dir)
    upload_path = Path(upload_dir)
    pdfs = [f for f in upload_path.iterdir() if f.suffix.lower() == '.pdf']

    with ProcessPoolExecutor() as executor:
        results = [executor.submit(process_pdf, pdf, root_archival, upload_dir) for pdf in pdfs]
        for future in results:
            future.result()  # This will wait for the process to complete and handle exceptions

In [ ]:
upload_directory = './upload'
archival_directory = './archival'
main(upload_directory, archival_directory)

In [ ]:
def move_files_and_prepare_download(subdir):
    """
    Move specific image files to designated directories and prepare the download folder using glob for pattern matching.
    """
    # Define the target directories for image files
    noteshrink_post_processed_dir = subdir / "noteshrinked_post_processed_images"
    noteshrinked_images_dir = subdir / "noteshrinked_images"
    noteshrink_post_processed_dir.mkdir(exist_ok=True)
    noteshrinked_images_dir.mkdir(exist_ok=True)

    # Process files in the preprocessed_images directory
    preprocessed_dir = subdir / "preprocessed_images"
    # Using glob to find matching files directly
    for file_path in preprocessed_dir.glob("ns_page*_post.png"):
        shutil.move(str(file_path), str(noteshrink_post_processed_dir / file_path.name))
    for file_path in preprocessed_dir.glob("ns_page*.png"):
        shutil.move(str(file_path), str(noteshrinked_images_dir / file_path.name))

    # Handle the PDF file moving to a new download directory
    download_dir = subdir.parent.parent / "download"
    download_dir.mkdir(exist_ok=True)
    pdf_target_dir = download_dir / subdir.name
    pdf_target_dir.mkdir(exist_ok=True)

    # Move noteshrinked.pdf if exists
    pdf_file = subdir / "preprocessed_images" / "noteshrinked.pdf"
    if pdf_file.exists():
        shutil.move(str(pdf_file), str(pdf_target_dir / pdf_file.name))

def process_archival_directory(archival_dir_path):
    """
    Process each subdirectory in archival concurrently using multithreading.
    """
    archival_dir = Path(archival_dir_path)
    subdirs = [d for d in archival_dir.iterdir() if d.is_dir()]

    with ThreadPoolExecutor() as executor:
        executor.map(move_files_and_prepare_download, subdirs)

if __name__ == "__main__":
    archival_directory = './archival'
    process_archival_directory(archival_directory)
